In [8]:
!pip install mplfinance

In [ ]:
!curl -L http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz -O && tar xzvf ta-lib-0.4.0-src.tar.gz
!cd ta-lib && ./configure --prefix=/usr && make && make install && cd - && pip install ta-lib

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   380  100   380    0     0   2288      0 --:--:-- --:--:-- --:--:--  2303
100   355  100   355    0     0   1098      0 --:--:-- --:--:-- --:--:--  1098
100 1299k  100 1299k    0     0  1486k      0 --:--:-- --:--:-- --:--:-- 3725k
ta-lib/
ta-lib/config.sub
ta-lib/aclocal.m4
ta-lib/CHANGELOG.TXT
ta-lib/include/
ta-lib/include/ta_abstract.h
ta-lib/include/ta_func.h
ta-lib/include/ta_common.h
ta-lib/include/ta_config.h.in
ta-lib/include/Makefile.am
ta-lib/include/ta_libc.h
ta-lib/include/ta_defs.h
ta-lib/missing
ta-lib/ta-lib.spec.in
ta-lib/config.guess
ta-lib/Makefile.in
ta-lib/ta-lib.dpkg.in
ta-lib/Makefile.am
ta-lib/autogen.sh
ta-lib/install-sh
ta-lib/configure
ta-lib/depcomp
ta-lib/HISTORY.TXT
ta-lib/configure.in
ta-lib/autom4te.cache/
ta-lib/autom4te.cache/output.0
ta-lib/autom4te.cache/requests
ta-lib/autom4te.cache/outpu

In [9]:
import pandas_datareader.data as pdr
def get_stock_data(code) :
  df = pdr.DataReader("{}.JP".format(code), "stooq").sort_index()
  return df

In [10]:
import numpy as np
import plotly.graph_objs as go
import talib as ta
import datetime as dt
import pandas as pd

In [11]:
def get_stock_graph(code, year, month, day) :
    df = get_stock_data(code)
    close =df["Close"]

    #移動平均線
    df["ma5"] = ta.SMA(close, timeperiod=5)
    df['ma25'] = ta.SMA(close, timeperiod=25)
    df["ma75"] = ta.SMA(close, timeperiod =75)
    df['ma100'] = ta.SMA(close, timeperiod=100)
    df["ma200"] = ta.SMA(df["Close"], timeperiod=200)

    #ゴールデンクロス、デッドクロス
    cross = df["ma5"] > df["ma25"]
    df["cross"] = cross
    cross_shift = cross.shift(1)
    temp_gc = (cross != cross_shift) & (cross ==True)
    temp_dc = (cross != cross_shift) & (cross == False)
    gc = [m if g == True else np.nan for g, m in zip(temp_gc, df['ma5'])]
    dc = [m if d == True else np.nan for d, m in zip(temp_dc, df['ma25'])]
    df["gc"], df["dc"] = gc, dc

    #ポリンジャーバンド
    df["upper2"], _, df["lower2"] = ta.BBANDS(close, timeperiod=25, nbdevup=2, nbdevdn=2, matype=ta.MA_Type.SMA)

    #MACD
    macd, macdsignal, hist = ta.MACD(close, fastperiod=12, slowperiod=26,signalperiod=9)
    df['macd'] = macd
    df["macd_signal"] = macdsignal
    df["hist"] = hist

    macdcross = df["macd"] > df["macd_signal"]
    df["macdcross"] = macdcross
    macdcross_shift = macdcross.shift(1)
    temp_macdgc = (macdcross != macdcross_shift) & (macdcross ==True)
    temp_macddc = (macdcross != macdcross_shift) & (macdcross == False)
    macdgc = [m if g == True else np.nan for g, m in zip(temp_macdgc, df['macd'])]
    macddc = [m if d == True else np.nan for d, m in zip(temp_macddc, df['macd_signal'])]
    df["macdgc"], df["macddc"] = macdgc, macddc

    #RSI
    df["rsi14"] = ta.RSI(close, 14)
    df["rsi28"] = ta.RSI(close, 28)
    df["70"], df["30"] = [70 for _ in close], [30 for _ in close]

    #ストキャスティクス
    slowK, slowD = ta.STOCH(df["High"], df["Low"], df["Close"],
                            fastk_period=5, slowk_period=3, slowk_matype=0,
                            slowd_period=3, slowd_matype=0)

    df["slowK"], df["slowD"] = slowK, slowD

    df["80"] = [80 for _ in close]
    df["20"] = [20 for _ in close]

    stochcross = df["slowK"] > df["slowD"]
    df["stochcross"] = stochcross
    stochcross_shift = stochcross.shift(1)
    temp_stochgc = (stochcross != stochcross_shift) & (stochcross ==True)
    temp_stochdc = (stochcross != stochcross_shift) & (stochcross == False)
    stochgc = [m if g == True else np.nan for g, m in zip(temp_stochgc, df['slowK'])]
    stochdc = [m if d == True else np.nan for d, m in zip(temp_stochdc, df['slowD'])]
    df["stochgc"], df["stochdc"] = stochgc, stochdc


    #--------------------------チャートパターン --------------------------
    #丸坊主
    marubozu = ta.CDLMARUBOZU(df["Open"], df["High"], df["Low"], df["Close"])
    df["marubozu_text"] = marubozu.replace({100:"買い", -100 : "売り", 0:""})
    df["marubozu_marker"] = (marubozu/100 * df["High"]).abs().replace({0:np.nan})

    #寄付坊主、大引坊主
    belthold = ta.CDLBELTHOLD(df["Open"], df["High"], df["Low"], df["Close"])
    df["belthold_text"] = belthold.replace({100:"買い", -100 : "売り", 0:""})
    df["belthold_marker"] = (belthold/100 * df["High"]).abs().replace({0:np.nan})

    #包み足
    engulfing = ta.CDLENGULFING(df["Open"], df["High"], df["Low"], df["Close"])
    df["engulfing_text"] = engulfing.replace({100:"買い", -100 : "売り", 0:""})
    df["engulfing_marker"] = (engulfing/100 * df["High"]).abs().replace({0:np.nan})

    #包み上げ/包み下げ
    outside = ta.CDL3OUTSIDE(df["Open"], df["High"], df["Low"], df["Close"])
    df["outside_text"] = outside.replace({100:"買い", -100 : "売り", 0:""})
    df["outside_marker"] = (outside/100 * df["High"]).abs().replace({0:np.nan})


    #はらみ足
    harami = ta.CDLHARAMI(df["Open"], df["High"], df["Low"], df["Close"])
    df["harami_text"] = harami.replace({100:"買い", -100 : "売り", 0:""})
    df["harami_marker"] = (harami/100 * df["High"]).abs().replace({0:np.nan})


    #はらみ上げ/はらみ下げ
    inside = ta.CDL3INSIDE(df["Open"], df["High"], df["Low"], df["Close"])
    df["inside_text"] = inside.replace({100:"買い", -100 : "売り", 0:""})
    df["inside_marker"] = (inside/100 * df["High"]).abs().replace({0:np.nan})




    #------------------------日付調整-------------------------------
    # rdf = df[dt.datetime(2021, 11, 1) : dt.datetime(2023, 3, 31)]
    rdf = df[dt.datetime(year, month, day):]
    #インデックスを文字列に
    rdf.index = pd.to_datetime(rdf.index).strftime("%m-%d-%Y")


    layout = {
        "height" : 1000,
        "title" : {"text" : "{}".format(code), "x" : 0.5},
        "xaxis" : {"rangeslider" : {"visible" : False}},
        "yaxis1" : {"domain":[.46,1.0], "title":"価格", "side":"left", "tickformat":","},
        "yaxis2" : {"domain":[.40, .46]},
        "yaxis3" : {"domain":[.30, .395], "title": "MACD", "side": "left"},
        "yaxis4" : {"domain":[.20, .295], "title": "RSI", "side": "left"},
        "yaxis5" : {"domain":[.10, .195], "title": "STC", "side": "left"},
        "yaxis6" : {"domain":[.00, .095], "title": "Volume", "side": "left"},
        "plot_bgcolor": "light blue"
        }

    data = [
        go.Candlestick(x=rdf.index, open=rdf["Open"], high = rdf["High"], low = rdf["Low"], close = rdf["Close"], name = "日足",
                      increasing_line_color = "red",
                      decreasing_line_color = "gray"),
        #移動平均線
        go.Scatter(yaxis="y1", x=rdf.index, y=rdf["ma5"], name = "MA5",
                  line = {"color": "royalblue", "width" : 1.2}),
        go.Scatter(yaxis="y1", x=rdf.index, y=rdf["ma25"], name = "MA25",
                  line = {"color": "lightseagreen", "width" : 1.2}),
        go.Scatter(yaxis="y1", x=rdf.index, y=rdf["ma75"], name = "MA75",
                  line = {"color": "yellow", "width" : 1.2}),
        go.Scatter(yaxis="y1", x=rdf.index, y=rdf["ma100"], name = "MA100",
                  line = {"color": "orange", "width" : 1.2}),
        go.Scatter(yaxis="y1", x=rdf.index, y=rdf["ma200"], name = "MA200",
                  line = {"color": "red", "width" : 1.2}),
        #ゴールデンクロス、デッドクロス
        go.Scatter(yaxis="y1", x=rdf.index, y=rdf["gc"], name="Golden Cross",
                  opacity = 0.5, mode="markers",
                  marker={"size":15, "color": "yellow"}),
        go.Scatter(yaxis="y1", x=rdf.index, y=rdf["dc"], name="Dead Cross",
                  opacity = 0.8, mode="markers",
                  marker={"size":15, "color": "black", "symbol":"x"}),
        #ポリンジャー
        go.Scatter(yaxis="y1", x=rdf.index, y=rdf["upper2"], name="",
                  line ={"color":"lavender", "width":0}),
        go.Scatter(yaxis="y1", x=rdf.index, y=rdf["lower2"], name="BB",
                  line ={"color":"lavender", "width":0},
                  fill="tonexty", fillcolor="rgba(170,170,170,.2)"),

        #MACD
        go.Scatter(yaxis="y3", x=rdf.index, y=rdf["macd"], name="macd",
                  line ={"color":"magenta", "width":1}),
        go.Scatter(yaxis="y3", x = rdf.index, y=rdf["macd_signal"], name = "signal",
                  line={"color": "green", "width":1}),
        go.Bar(yaxis="y3", x=rdf.index, y=rdf["hist"], name="histgram",
              marker={"color":"slategray"}),

        go.Scatter(yaxis="y3", x=rdf.index, y=rdf["macdgc"], name="MACD Golden Cross",
                  opacity = 0.5, mode="markers",
                  marker={"size":15, "color": "yellow"}),
        go.Scatter(yaxis="y3", x=rdf.index, y=rdf["macddc"], name="MACD Dead Cross",
                  opacity = 0.8, mode="markers",
                  marker={"size":15, "color": "black", "symbol":"x"}),

        #RSI
        go.Scatter(yaxis="y4", x=rdf.index, y=rdf["rsi14"], name="RSI14",
                  line={"color":"magenta", "width":1}),
        go.Scatter(yaxis="y4", x=rdf.index, y=rdf["rsi28"], name="RSI14",
                  line ={"color":"green", "width":1}),

        #補助線
        go.Scatter(yaxis="y4", x=rdf.index, y=rdf["30"], name="30",
                  line={"color":"black", "width":0.5}),
        go.Scatter(yaxis="y4", x=rdf.index, y=rdf["70"], name="70",
                  line={"color":"black", "width":0.5}),

        #ストキャスティクス
        go.Scatter(yaxis="y5", x=rdf.index, y=rdf["slowK"], name="slowK",
                  line={"color":"magenta", "width":1}),
        go.Scatter(yaxis="y5", x=rdf.index, y=rdf["slowD"], name="slowD",
                line={"color":"magenta", "width":1}),

        go.Scatter(yaxis="y5", x=rdf.index, y=rdf["stochgc"], name="STOCH Golden Cross",
                  opacity = 0.5, mode="markers",
                  marker={"size":15, "color": "yellow"}),
        go.Scatter(yaxis="y5", x=rdf.index, y=rdf["stochdc"], name="STOCH Dead Cross",
                  opacity = 0.8, mode="markers",
                  marker={"size":15, "color": "black", "symbol":"x"}),

        #補助線
        go.Scatter(yaxis="y5", x=rdf.index, y=rdf["20"], name="20",
                  line={"color":"black", "width":0.5}),
        go.Scatter(yaxis="y5", x=rdf.index, y=rdf["80"], name="80",
                  line={"color":"black", "width":0.5}),


        #-------------チャートパターン----------------
        #丸坊主
        go.Scatter(x=rdf.index,
                  y=rdf["marubozu_marker"],
                  mode = "markers+text",
                  text=rdf["marubozu_text"],
                  textposition="top center",
                  name = "丸坊主",
                  marker = {"size":12, "color":"blue", "opacity":0.9},
                  textfont = {"size":14, "color":"black"}),

        #寄付坊主、大引坊主
        go.Scatter(x=rdf.index,
                y=rdf["belthold_marker"],
                mode = "markers+text",
                text=rdf["belthold_text"],
                textposition="top center",
                name = "寄付坊主/大引坊主",
                marker = {"size":12, "color":"blue", "opacity":0.4},
                textfont = {"size":14, "color":"black"}),

        #包み足
        go.Scatter(x=rdf.index,
            y=rdf["engulfing_marker"],
            mode = "markers+text",
            text=rdf["engulfing_text"],
            textposition="top center",
            name = "包み足",
            marker = {"size":12, "color":"green", "opacity":0.4},
            textfont = {"size":14, "color":"black"}),

        #包み上げ/包み下げ
        go.Scatter(x=rdf.index,
            y=rdf["outside_marker"],
            mode = "markers+text",
            text=rdf["outside_text"],
            textposition="top center",
            name = "包み上げ/包み下げ",
            marker = {"size":12, "color":"green", "opacity":0.9},
            textfont = {"size":14, "color":"black"}),

        #はらみ足
        go.Scatter(x=rdf.index,
            y=rdf["harami_marker"],
            mode = "markers+text",
            text=rdf["harami_text"],
            textposition="top center",
            name = "はらみ足",
            marker = {"size":12, "color":"purple", "opacity":0.4},
            textfont = {"size":14, "color":"black"}),

        #はらみ上げ/はらみ下げ
        go.Scatter(x=rdf.index,
            y=rdf["inside_marker"],
            mode = "markers+text",
            text=rdf["inside_text"],
            textposition="top center",
            name = "はらみ上げ/はらみ下げ",
            marker = {"size":12, "color":"purple", "opacity":0.9},
            textfont = {"size":14, "color":"black"}),


        #出来高
        go.Bar(yaxis="y6", x=rdf.index, y=rdf["Volume"], name ="Volume",
                  marker={"color":"slategray" })
        ]


    fig = go.Figure(data = data, layout = go.Layout(layout))

    fig.update_layout({
        "xaxis": {
            "ticktext" : [x.split("-")[0] + "-" + x.split("-")[1] for x in rdf.index[::20]],
            "tickvals" : [x for x in rdf.index[::20]]
        }
    })

    f = go.Figure(fig)
    f.show()

    return

In [15]:
get_stock_graph(5574, 2020, 3, 1)